In [1]:
import torch
from fancy_einsum import einsum

In [2]:
TOPK = 7

In [3]:
# n_experts = 2
# topk (capacity eksperta) = 3
# gate_dim = 4
x = torch.randint(0, 4, (3, 4, 5)) # (batch, cutoff, dmodel)
gate = torch.randint(0, 4, (4, 2)) # (cutoff, n_experts)
print(f'x:\n{x}\n\ngate:\n{gate}')

x:
tensor([[[0, 3, 0, 2, 2],
         [1, 1, 1, 3, 2],
         [2, 1, 1, 1, 2],
         [1, 0, 2, 3, 2]],

        [[1, 1, 0, 0, 0],
         [0, 1, 0, 2, 1],
         [1, 2, 0, 2, 2],
         [0, 3, 1, 0, 1]],

        [[2, 1, 3, 0, 0],
         [2, 2, 1, 0, 0],
         [1, 1, 3, 0, 1],
         [2, 0, 1, 1, 3]]])

gate:
tensor([[3, 1],
        [0, 3],
        [2, 3],
        [1, 0]])


In [4]:
# 1. używamy gate, żeby stworzyć reprezentację ekspertów zamiast dmodel
gate_out = einsum('batch cutoff dmodel, cutoff n_experts -> batch cutoff n_experts', x, gate)
print(f'gate_out:\n{gate_out}\n\n shape: {gate_out.shape}')

gate_out:
tensor([[[21,  7],
         [ 0, 24],
         [14, 21],
         [ 8,  0]],

        [[ 6,  2],
         [ 0, 12],
         [14, 21],
         [ 5,  0]],

        [[18,  6],
         [ 0, 15],
         [12, 18],
         [ 7,  0]]])

 shape: torch.Size([3, 4, 2])


In [5]:
# wymiar eksperta chcę żeby był na początku
gate_out = gate_out.permute(2, 0, 1)
gate_out

tensor([[[21,  0, 14,  8],
         [ 6,  0, 14,  5],
         [18,  0, 12,  7]],

        [[ 7, 24, 21,  0],
         [ 2, 12, 21,  0],
         [ 6, 15, 18,  0]]])

In [6]:
gate_out.shape

torch.Size([2, 3, 4])

In [7]:
# spłaszczam wymiary 2 i 3, żeby każdy ekspert wybrał sobie swobodnie tokeny (ogólnie tokenów w batchu jest 12)
gate_out = gate_out.flatten(start_dim=1)
gate_out

tensor([[21,  0, 14,  8,  6,  0, 14,  5, 18,  0, 12,  7],
        [ 7, 24, 21,  0,  2, 12, 21,  0,  6, 15, 18,  0]])

In [8]:
gate_out.shape

torch.Size([2, 12])

In [9]:
# make gate_out float tensor
gate_out = gate_out.float()

In [10]:
# teraz przeprowadzamy softmax po 1 wymiarze
gate_out = gate_out.softmax(dim=1)

In [11]:
gate_out

tensor([[9.5081e-01, 7.2096e-10, 8.6702e-04, 2.1491e-06, 2.9085e-07, 7.2096e-10,
         8.6702e-04, 1.0700e-07, 4.7338e-02, 7.2096e-10, 1.1734e-04, 7.9062e-07],
        [3.7561e-08, 9.0729e-01, 4.5171e-02, 3.4251e-11, 2.5309e-10, 5.5746e-06,
         4.5171e-02, 3.4251e-11, 1.3818e-08, 1.1197e-04, 2.2489e-03, 3.4251e-11]])

In [12]:
softmax_gate_out = gate_out
# wybierz topk tokenów dla każdego eksperta
gate_out = torch.topk(gate_out, k=TOPK, dim=1).indices

gate_out

tensor([[ 0,  8,  6,  2, 10,  3, 11],
        [ 1,  6,  2, 10,  9,  5,  0]])

In [16]:
print(f'shape of softmax gate out: {softmax_gate_out.shape}, shape of x: {x.shape}')

shape of softmax gate out: torch.Size([2, 12]), shape of x: torch.Size([3, 4, 5])


In [19]:
gate_out = gate_out.flatten()

In [17]:
x = x.flatten(start_dim=0, end_dim=1)

In [20]:
print(f'shape of softmax gate out: {softmax_gate_out.shape}, shape of x: {x.shape}')

shape of softmax gate out: torch.Size([2, 12]), shape of x: torch.Size([12, 5])


In [21]:
torch.index_select(softmax_gate_out, dim=1, index=gate_out)

tensor([[9.5081e-01, 4.7338e-02, 8.6702e-04, 8.6702e-04, 1.1734e-04, 2.1491e-06,
         7.9062e-07, 7.2096e-10, 8.6702e-04, 8.6702e-04, 1.1734e-04, 7.2096e-10,
         7.2096e-10, 9.5081e-01],
        [3.7561e-08, 1.3818e-08, 4.5171e-02, 4.5171e-02, 2.2489e-03, 3.4251e-11,
         3.4251e-11, 9.0729e-01, 4.5171e-02, 4.5171e-02, 2.2489e-03, 1.1197e-04,
         5.5746e-06, 3.7561e-08]])

In [16]:
gate_out = gate_out.flatten()

In [13]:
gate_out

tensor([[ 0,  8,  6,  2, 10,  3, 11],
        [ 1,  6,  2, 10,  9,  5,  0]])

In [14]:
x

tensor([[[0, 3, 0, 2, 2],
         [1, 1, 1, 3, 2],
         [2, 1, 1, 1, 2],
         [1, 0, 2, 3, 2]],

        [[1, 1, 0, 0, 0],
         [0, 1, 0, 2, 1],
         [1, 2, 0, 2, 2],
         [0, 3, 1, 0, 1]],

        [[2, 1, 3, 0, 0],
         [2, 2, 1, 0, 0],
         [1, 1, 3, 0, 1],
         [2, 0, 1, 1, 3]]])

In [15]:
x.shape

torch.Size([3, 4, 5])

In [20]:
torch.index_select(x, dim=0, index=gate_out)

tensor([[3, 2, 3, 2, 2],
        [2, 0, 1, 2, 3],
        [2, 0, 3, 0, 2],
        [3, 3, 0, 1, 0],
        [0, 3, 1, 0, 1],
        [0, 2, 0, 1, 0],
        [3, 3, 1, 1, 1],
        [3, 2, 3, 2, 2],
        [3, 3, 1, 1, 1],
        [1, 3, 2, 2, 0],
        [3, 2, 1, 1, 1],
        [1, 0, 2, 3, 0],
        [0, 2, 2, 0, 2],
        [0, 3, 1, 0, 1]])

In [15]:
x

tensor([[1, 3, 2, 2, 0],
        [0, 2, 0, 1, 0],
        [3, 3, 0, 1, 0],
        [3, 3, 1, 1, 1],
        [1, 0, 2, 3, 0],
        [0, 3, 1, 0, 1],
        [2, 0, 1, 2, 3],
        [1, 0, 0, 2, 0],
        [3, 2, 1, 1, 1],
        [3, 2, 3, 2, 2],
        [2, 0, 3, 0, 2],
        [0, 2, 2, 0, 2]])

In [23]:
torch.arange(12).unsqueeze(1)

tensor([[ 0],
        [ 1],
        [ 2],
        [ 3],
        [ 4],
        [ 5],
        [ 6],
        [ 7],
        [ 8],
        [ 9],
        [10],
        [11]])

In [25]:
gate_out.unsqueeze(0)

tensor([[ 0,  8,  6,  2, 10,  3, 11,  1,  6,  2, 10,  9,  5,  0]])

In [30]:
torch.where(gate_out.unsqueeze(0) == torch.arange(12).unsqueeze(1))[0]

tensor([ 0,  0,  1,  2,  2,  3,  5,  6,  6,  8,  9, 10, 10, 11])

In [31]:
gate_out.unsqueeze(0) == torch.arange(12).unsqueeze(1)

tensor([[ True, False, False, False, False, False, False, False, False, False,
         False, False, False,  True],
        [False, False, False, False, False, False, False,  True, False, False,
         False, False, False, False],
        [False, False, False,  True, False, False, False, False, False,  True,
         False, False, False, False],
        [False, False, False, False, False,  True, False, False, False, False,
         False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False,  True, False],
        [False, False,  True, False, False, False, False, False,  True, False,
         False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False],
        [False,  True, False, False, False, False, False, False,

In [293]:
# zakoduj wybór eksperta jako one hot
# tutaj dodać i odjąć odpowiedni softmax (można jakoś pomnożyć przez one te softmaxy i wtedy dodać i odjąć takie zwierzę)
gate_out = torch.nn.functional.one_hot(gate_out, num_classes=12)
gate_out

tensor([[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]],

        [[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]])

In [294]:
print(f'Gate out shape: {gate_out.shape}, softmax_gate_out shape: {softmax_gate_out.shape}')

Gate out shape: torch.Size([2, 7, 12]), softmax_gate_out shape: torch.Size([2, 12])


In [308]:
softmax_mask = einsum('n_experts topk n_examples, n_experts n_examples -> n_experts topk n_examples', gate_out, softmax_gate_out)
softmax_mask

tensor([[[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          9.0924e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 4.5268e-02, 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 4.5268e-02,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.1221e-04,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [1.1221e-04, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 1.3848e-08, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+0

In [310]:
gate_out = gate_out + softmax_mask - softmax_mask
print(f'Gate out: {gate_out} \n\n softmax_mask: {softmax_mask}')

Gate out: tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          1.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 1.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 1.0000, 0.0000]],

        [[0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.

In [248]:
gate_out_flattened = gate_out.flatten(start_dim=0, end_dim=1)
print(f'gate_out:\n{gate_out_flattened}\n\n shape: {gate_out_flattened.shape}')

gate_out:
tensor([[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]])

 shape: torch.Size([14, 12])


In [249]:
id = torch.eye(12)
id

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [250]:
chosen_examples = einsum('n_elems n_elems, expert_layer_width n_elems -> expert_layer_width n_elems', id, gate_out_flattened)
print(f'chosen_examples:\n{chosen_examples}\n\n shape: {chosen_examples.shape}')

chosen_examples:
tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])

 shape: torch.Size([14, 12])


In [251]:
not_chosen_examples = (chosen_examples.sum(dim=0) == 0).float()
not_chosen_examples

tensor([0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.])

In [252]:
gate_out.shape

torch.Size([2, 7, 12])

In [253]:
# z powrotem odwróć spłaszczenie
# gate_out = gate_out.view(2, TOPK, 3, 4)
gate_out

tensor([[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
         [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

        [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
         [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]])

In [254]:
# batch_size=3, cutoff=4

In [255]:
x

tensor([[[0, 1, 0, 3, 3],
         [3, 2, 3, 0, 1],
         [3, 1, 1, 0, 0],
         [0, 2, 2, 1, 1]],

        [[1, 3, 1, 2, 1],
         [1, 1, 1, 3, 0],
         [1, 3, 3, 1, 3],
         [1, 1, 2, 1, 3]],

        [[3, 0, 3, 0, 1],
         [0, 1, 1, 0, 3],
         [1, 3, 1, 0, 3],
         [2, 1, 0, 2, 3]]])

In [256]:
x.shape

torch.Size([3, 4, 5])

In [14]:
x = x.flatten(start_dim=0, end_dim=1)
x
# tu zapisać x przed wejściem do layera

tensor([[1, 3, 2, 2, 0],
        [0, 2, 0, 1, 0],
        [3, 3, 0, 1, 0],
        [3, 3, 1, 1, 1],
        [1, 0, 2, 3, 0],
        [0, 3, 1, 0, 1],
        [2, 0, 1, 2, 3],
        [1, 0, 0, 2, 0],
        [3, 2, 1, 1, 1],
        [3, 2, 3, 2, 2],
        [2, 0, 3, 0, 2],
        [0, 2, 2, 0, 2]])

In [258]:
x_before_experts = x

In [259]:
# teraz permutujemy macierz x zgodne z tym one hot
x = einsum('n_elems dmodel, n_experts topk n_elems -> n_experts topk dmodel', x, gate_out)
print(f'x:\n{x}\n\n shape: {x.shape}')

x:
tensor([[[1, 3, 1, 2, 1],
         [0, 1, 0, 3, 3],
         [3, 0, 3, 0, 1],
         [1, 1, 2, 1, 3],
         [2, 1, 0, 2, 3],
         [0, 2, 2, 1, 1],
         [3, 2, 3, 0, 1]],

        [[3, 2, 3, 0, 1],
         [1, 1, 1, 3, 0],
         [0, 1, 1, 0, 3],
         [2, 1, 0, 2, 3],
         [1, 1, 2, 1, 3],
         [0, 2, 2, 1, 1],
         [1, 3, 1, 0, 3]]])

 shape: torch.Size([2, 7, 5])


In [260]:
# wypłasczamy wymiar 0 i 1, tak żeby otrzymać macierz wejścia do warstwy ekspertóœ
x = x.flatten(start_dim=0, end_dim=1)
print(f'x:\n{x}\n\n shape: {x.shape}')

x:
tensor([[1, 3, 1, 2, 1],
        [0, 1, 0, 3, 3],
        [3, 0, 3, 0, 1],
        [1, 1, 2, 1, 3],
        [2, 1, 0, 2, 3],
        [0, 2, 2, 1, 1],
        [3, 2, 3, 0, 1],
        [3, 2, 3, 0, 1],
        [1, 1, 1, 3, 0],
        [0, 1, 1, 0, 3],
        [2, 1, 0, 2, 3],
        [1, 1, 2, 1, 3],
        [0, 2, 2, 1, 1],
        [1, 3, 1, 0, 3]])

 shape: torch.Size([14, 5])


In [261]:
# dodajemy do siebie wyniki dla tych tokenów, które były wybrane przez wielu ekspertów
# upewnić się, czy odpermutowuję tu poprawnie
chosen_examples.shape

torch.Size([14, 12])

In [262]:
summed = einsum('expert_layer_width n_elems, expert_layer_width dmodel -> n_elems dmodel', chosen_examples.float(), x.float())
print(f'summed:\n{summed}\n\n shape: {summed.shape}')

summed:
tensor([[0., 1., 0., 3., 3.],
        [6., 4., 6., 0., 2.],
        [0., 0., 0., 0., 0.],
        [0., 4., 4., 2., 2.],
        [1., 3., 1., 2., 1.],
        [1., 1., 1., 3., 0.],
        [0., 0., 0., 0., 0.],
        [2., 2., 4., 2., 6.],
        [3., 0., 3., 0., 1.],
        [0., 1., 1., 0., 3.],
        [1., 3., 1., 0., 3.],
        [4., 2., 0., 4., 6.]])

 shape: torch.Size([12, 5])


In [264]:
# add examples that were not chosen by any expert
einsum('n_elems dmodel, n_elems -> n_elems dmodel', x_before_experts, not_chosen_examples)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [3., 1., 1., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [1., 3., 3., 1., 3.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [265]:
summed += einsum('n_elems dmodel, n_elems -> n_elems dmodel', x_before_experts, not_chosen_examples)
print(f'summed:\n{summed}\n\n shape: {summed.shape}')

summed:
tensor([[0., 1., 0., 3., 3.],
        [6., 4., 6., 0., 2.],
        [3., 1., 1., 0., 0.],
        [0., 4., 4., 2., 2.],
        [1., 3., 1., 2., 1.],
        [1., 1., 1., 3., 0.],
        [1., 3., 3., 1., 3.],
        [2., 2., 4., 2., 6.],
        [3., 0., 3., 0., 1.],
        [0., 1., 1., 0., 3.],
        [1., 3., 1., 0., 3.],
        [4., 2., 0., 4., 6.]])

 shape: torch.Size([12, 5])


In [266]:
# again reshape summed to original shape with batch and cutoff
summed = summed.view(3, 4, 5)
print(f'summed:\n{summed}\n\n shape: {summed.shape}')

summed:
tensor([[[0., 1., 0., 3., 3.],
         [6., 4., 6., 0., 2.],
         [3., 1., 1., 0., 0.],
         [0., 4., 4., 2., 2.]],

        [[1., 3., 1., 2., 1.],
         [1., 1., 1., 3., 0.],
         [1., 3., 3., 1., 3.],
         [2., 2., 4., 2., 6.]],

        [[3., 0., 3., 0., 1.],
         [0., 1., 1., 0., 3.],
         [1., 3., 1., 0., 3.],
         [4., 2., 0., 4., 6.]]])

 shape: torch.Size([3, 4, 5])
